# Sentiment Analysis Framework

## 1. Installing Dependencies

In [9]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [10]:
import sys
print(sys.path)

['E:\\Sentiment_Analysis_Using_NLP', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311', '', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\win32', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\Pythonwin']


In [11]:
import torch

In [12]:
!pip install transformers

In [13]:
import transformers

In [14]:
!pip install requests 

In [15]:
import requests

In [16]:
!pip install beautifulsoup4 

In [17]:
import bs4

In [18]:
!pip install pandas 

In [34]:
import pandas as pd

In [20]:
!pip install numpy

In [35]:
import numpy as np

## 2. Import Statements

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## 3. Model

### To know more about the used model 
#### Log on to ->> https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

In [23]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Sentiments Calculation

In [24]:
tokens = tokenizer.encode('I hate doing this, this is torture', return_tensors='pt')

In [25]:
tokens

tensor([[  101,   151, 39487, 27799, 10372,   117, 10372, 10127, 57238,   102]])

In [27]:
tokenizer.decode(tokens[0])
# decode() -> can't pass list
# therfore tokens[0]

'[CLS] i hate doing this, this is torture [SEP]'

In [28]:
result = model(tokens)

In [29]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.9811,  0.3842, -1.1437, -1.6661, -0.2913]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

### Understanding the Results

Each number in the logits tensor represents the model's confidence score for each class.

The number of elements in the logits tensor matches the number of classes in our classification task.

To interpret the logits, we apply the softmax function to convert them into probabilities.

The class with the highest probability is considered the predicted class.


In [30]:
result.logits

tensor([[ 2.9811,  0.3842, -1.1437, -1.6661, -0.2913]],
       grad_fn=<AddmmBackward0>)

In [32]:
# it should return 1st value as it is the highest
torch.argmax(result.logits)

tensor(0)

In [33]:
int(torch.argmax(result.logits))

0

## Sentiment Rating Interpretation

In our sentiment analysis output, we obtain a range from 0 to 4:

- **0**: Lowest sentiment
- **4**: Highest sentiment

However, as humans, we typically interpret ratings on a scale from 1 to 5:

- **1**: Lowest sentiment
- **5**: Highest sentiment

This adjustment helps in aligning the sentiment analysis results with human interpretation.


# 4. Scraping Reviews
#### plz dont do scraping its bad and illegal
## USE APIs

In [36]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [37]:
reviews

['The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I

# 5. Load Reviews into DataFrame and Score

In [38]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [40]:
df.head()

,review
0,The food is fresh and tasty. The scallop cevi...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...


In [39]:
df['review'].iloc[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

## We have the data now we will do exactly what we did in STEP 3

## Creating a Sentiment Function

We're going to create a function that streamlines the sentiment analysis process. This function will make it easier to analyze sentiment for multiple strings.

### What does the function do?

The function encapsulates the entire sentiment analysis pipeline. You pass a string to the function, and it returns the sentiment score for that string.

### Why is this useful?

By wrapping the sentiment analysis process in a function:
- We make the code reusable. Instead of rewriting the sentiment analysis steps each time we want to analyze a string, we can simply call the function.
- It simplifies the code. We don't need to worry about the details of each step in the sentiment analysis pipeline every time we want to analyze sentiment.

### How does it work?

1. **Input**: The function takes a string as input.
2. **Preprocessing**: It preprocesses the input string, preparing it for sentiment analysis.
3. **Sentiment Analysis**: It applies sentiment analysis to the preprocessed string, determining the sentiment score.
4. **Output**: Finally, it returns the sentiment score for the input string.

By encapsulating these steps in a function, we can easily analyze sentiment for multiple strings without repeating code.


In [41]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))

In [43]:
sentiment_score(df['review'].iloc[0])

3

In [44]:
sentiment_score(df['review'].iloc[1])

1

In [48]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
# tokens limit : 512

In [49]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,3
1,The food was decent not great.. We had the gu...,1
2,"Food was okay, guacamole was below average. Se...",1
3,The food and service here was really good. It...,4
4,Visiting from Texas and decided to give this r...,4
5,Don't come here expecting legit Mexican food b...,2
6,Out of all the restaurants that I tried in Syd...,4
7,"Great atmosphere, attentive service, solid mar...",2
8,We came here on a Thursday night @ 5pm and by ...,3
9,Have been here twice and have absolutely loved...,4


In [50]:
df['review']

0    The food is fresh and tasty.  The scallop cevi...
1    The food was decent not great..  We had the gu...
2    Food was okay, guacamole was below average. Se...
3    The food and service here was really good.  It...
4    Visiting from Texas and decided to give this r...
5    Don't come here expecting legit Mexican food b...
6    Out of all the restaurants that I tried in Syd...
7    Great atmosphere, attentive service, solid mar...
8    We came here on a Thursday night @ 5pm and by ...
9    Have been here twice and have absolutely loved...
Name: review, dtype: object